## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [109]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')
df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1


In [110]:
# View the column names in the data
print(df.columns)

Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')


In [111]:
# Use the describe function to gather some basic statistics
print(df.describe())

          client_id      order_id    order_week    order_year    unit_price  \
count  54639.000000  5.463900e+04  54639.000000  54639.000000  54639.000000   
mean   54837.869416  5.470190e+06     11.359139   2022.993064    136.267207   
std    25487.438231  2.599807e+06      7.023499      0.082997    183.873135   
min    10033.000000  1.000886e+06      1.000000   2022.000000      0.010000   
25%    33593.000000  3.196372e+06      6.000000   2023.000000     20.800000   
50%    53305.000000  5.496966e+06     11.000000   2023.000000     68.310000   
75%    78498.000000  7.733869e+06     17.000000   2023.000000    173.160000   
max    99984.000000  9.998480e+06     52.000000   2023.000000   1396.230000   

          unit_cost   unit_weight           qty   line_number  
count  54639.000000  54639.000000  5.463900e+04  54639.000000  
mean      99.446073      5.004116  5.702646e+02      2.979667  
std      133.164267      5.326599  1.879552e+04      2.436320  
min        0.010000      0.00000

In [112]:
# Check for missing values in key columns
print(df[['unit_price', 'qty', 'unit_weight']].isnull().sum())

# Convert columns to appropriate data types
df['unit_price'] = pd.to_numeric(df['unit_price'], errors='coerce')
df['qty'] = pd.to_numeric(df['qty'], errors='coerce')
df['unit_weight'] = pd.to_numeric(df['unit_weight'], errors='coerce')
df['order_id'] = pd.to_numeric(df['order_id'], errors='coerce')

# Remove currency symbols and commas from price columns if they exist
for col in ['unit_price', 'unit_cost']:
    if df[col].dtype == 'object':
        df[col] = df[col].astype(str).str.replace('$', '').str.replace(',', '').astype(float)

 # Check for and remove duplicates
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")
if duplicates > 0:
    df = df.drop_duplicates()




unit_price     0
qty            0
unit_weight    0
dtype: int64
Number of duplicate rows: 0


In [113]:
# Use this space to do any additional research
# and familiarize yourself with the data.

# Count orders by year
year_counts = df['order_year'].value_counts().sort_index()
print("Orders by year:")
print(year_counts)

# Count orders by week
week_counts = df['order_week'].value_counts().sort_index()
print("Orders by week:")
print(week_counts)

# Count orders by category
category_counts = df['category'].value_counts().sort_index()
print ("Category counts:")
print (category_counts)



Orders by year:
order_year
2022      379
2023    54260
Name: count, dtype: int64
Orders by week:
order_week
1     2426
2     2837
3     2659
4     2537
5     2541
6     2628
7     2680
8     2641
9     2448
10    2475
11    2483
12    2402
13    2393
14    2523
15    2493
16    2713
17    2662
18    2363
19    2698
20    2338
21    2645
22     675
52     379
Name: count, dtype: int64
Category counts:
category
consumables    23538
decor           7954
electronics     2832
furniture      11915
software        8400
Name: count, dtype: int64


In [114]:
# What three item categories had the most entries?

# Group by item category and count the number of orders
category_counts = df.groupby('category')['order_id'].count().sort_values(ascending=False)

# Get only the top 3 categories
top_3_categories = category_counts.head(3)

# Display the top 3 category counts
print("Top 3 categories by number of orders:")
print(top_3_categories)

Top 3 categories by number of orders:
category
consumables    23538
furniture      11915
software        8400
Name: order_id, dtype: int64


In [115]:
# For the category with the most entries, which subcategory had the most entries?

top_category = df.groupby('category')['order_id'].count().sort_values(ascending=False).index[0]
print(f"The category with the most orders is: {top_category}")

# Filter the dataframe to only include rows from the top category
top_category_df = df[df['category'] == top_category]

# Group by subcategory and count orders
subcategory_counts = top_category_df.groupby('subcategory')['order_id'].count().sort_values(ascending=False)

# Display the subcategory counts
print(f"\nNumber of orders by subcategory within '{top_category}':")
print(subcategory_counts)

The category with the most orders is: consumables

Number of orders by subcategory within 'consumables':
subcategory
bathroom supplies    6424
paper products       5874
kitchen supplies     5746
misc                 2891
pens                 2603
Name: order_id, dtype: int64


In [129]:
# Which five clients had the most entries in the data?

# Count the number of entries for each client
client_entry_counts = df['client_id'].value_counts()

# Get the top 5 clients by number of entries
top_5_client_ids = client_entry_counts.head(5).index.tolist()

# Create a DataFrame with detailed information for the top 5 clients
top_5_clients_summary = df[df['client_id'].isin(top_5_client_ids)].groupby('client_id').agg({
    'first': 'first',  # Get the first name for each client
    'last': 'first',   # Get the last name for each client
    'client_id': 'size'  # Count the number of entries
}).rename(columns={'client_id': 'Number of Entries'}).reset_index()

# Sort by number of entries in descending order
top_5_clients_summary = top_5_clients_summary.sort_values('Number of Entries', ascending=False)

# Rename columns for clarity
top_5_clients_summary.columns = ['Client ID', 'First Name', 'Last Name', 'Number of Entries']

# Print the results
print("Top 5 Clients by Number of Entries:")
print(top_5_clients_summary)



Top 5 Clients by Number of Entries:
   Client ID First Name Last Name  Number of Entries
1      33615    Jessica     Reyes                220
4      66037     Angela   Everett                211
3      46820      Bryan     Myers                209
0      24741     Kendra   Garrett                207
2      38378  Alexandra     Young                207


In [117]:
# Extract and store the client ids of those top 5 clients in a list.
top_5_clients = client_order_counts.head(5).index.tolist()

# Print the list to verify
print("List of top 5 client IDs:")
print(top_5_clients)



List of top 5 client IDs:
[33615, 66037, 46820, 38378, 24741]


In [130]:
# How many total units (the qty column) did the client with the most entries order?
# Identify the client with the most orders
top_client_id = client_order_counts.index[0]
print(f"Client ID with the most orders: {top_client_id}")

# Filter dataframe to only include rows for top client
top_client_df = df[df['client_id'] == top_client_id]

# Sum the qty column to get the total units ordered by top client
total_units = top_client_df['qty'].sum()

print(f"Total units ordered by client {top_client_id}: {total_units}")

Client ID with the most orders: 33615
Total units ordered by client 33615: 64313


## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [119]:
# Create a column that calculates the subtotal for each line using the unit_price and the qty
df['subtotal'] = df['unit_price'] * df['qty']

# Display the first few rows to verify the calculation
df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number,subtotal
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1,115164.00
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0,523.95
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6,527.28
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3,1056.18
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1,3902.00


In [131]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound for orders over 50 pounds and $10 per pound for items 50 pounds or under.

# Define a function to calculate shipping price based on weight
def calculate_shipping_price(weight):
    if weight > 50:
        return weight * 7  # $7 per pound for orders over 50 pounds
    else:
        return weight * 10  # $10 per pound for orders 50 pounds or under

# Create column for total weight
df['total_weight'] = df['qty'] * df['unit_weight']

# Apply the function to each row's total weight
df['shipping_price'] = df['total_weight'].apply(calculate_shipping_price)


# Display the results to verify
print(df[['unit_weight', 'qty', 'total_weight', 'shipping_price']].head())


   unit_weight  qty  total_weight  shipping_price
0         7.50  105        787.50         5512.50
1         1.49   21         31.29          312.90
2         1.68   39         65.52          458.64
3         1.23   29         35.67          356.70
4        46.43   20        928.60         6500.20


In [132]:
# Create a column for the total price using the subtotal and the shipping price along with a sales tax of 9.25%


# Calculate tax on BOTH subtotal and shipping
tax_rate = 0.0925
df['taxable_amount'] = df['subtotal'] + df['shipping_price']
df['sales_tax'] = df['taxable_amount'] * tax_rate

df['total_price'] = df['subtotal'] + df['shipping_price'] + df['sales_tax']

# Display the breakdown for verification
print(df[['subtotal', 'shipping_price', 'sales_tax', 'total_price']].head())




    subtotal  shipping_price     sales_tax    total_price
0  115164.00         5512.50  11162.576250  131839.076250
1     523.95          312.90     77.408625     914.258625
2     527.28          458.64     91.197600    1077.117600
3    1056.18          356.70    130.691400    1543.571400
4    3902.00         6500.20    962.203500   11364.403500


In [133]:
# Create a column for the cost of each line using unit cost, qty, and
# shipping price (assume the shipping cost is exactly what is charged to the client).

df['total_cost'] = (df['unit_cost'] * df['qty']) + df['shipping_price']

# Display the first few rows to verify the calculation
df.head()


,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,qty,line_number,subtotal,total_weight,shipping_price,taxable_amount,sales_tax,total_price,total_cost,profit
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,105,1,115164.00,787.50,5512.50,120676.50,11162.576250,131839.076250,85597.05,46242.026250
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,21,0,523.95,31.29,312.90,836.85,77.408625,914.258625,629.79,284.468625
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,39,6,527.28,65.52,458.64,985.92,91.197600,1077.117600,765.18,311.937600
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,29,3,1056.18,35.67,356.70,1412.88,130.691400,1543.571400,1077.35,466.221400
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,20,1,3902.00,928.60,6500.20,10402.20,962.203500,11364.403500,8663.60,2700.803500


In [134]:
# Create a column for the profit of each line using line cost and line price

df['profit'] = df['total_price'] - df['total_cost']



# Display the first few rows to verify the calculation
df.head()


,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,qty,line_number,subtotal,total_weight,shipping_price,taxable_amount,sales_tax,total_price,total_cost,profit
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,105,1,115164.00,787.50,5512.50,120676.50,11162.576250,131839.076250,85597.05,46242.026250
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,21,0,523.95,31.29,312.90,836.85,77.408625,914.258625,629.79,284.468625
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,39,6,527.28,65.52,458.64,985.92,91.197600,1077.117600,765.18,311.937600
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,29,3,1056.18,35.67,356.70,1412.88,130.691400,1543.571400,1077.35,466.221400
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,20,1,3902.00,928.60,6500.20,10402.20,962.203500,11364.403500,8663.60,2700.803500


## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [135]:
# Check your work using the totals above

# Check Order ID 2742071

def check_order_total(df, order_id, receipt_total):
    # Sum total_price for all line items in this order
    order_lines = df[df['order_id'] == order_id]
    
    
    # Calculate the order total
    calculated_total = order_lines['total_price'].sum()
    
    # Compare to receipt
    match_status = "✓ MATCH" if abs(calculated_total - receipt_total) < 0.01 else "✗ MISMATCH"
    
    return (f"Order ID {order_id}:\n"
            f"  Calculated Total: ${calculated_total:,.2f}\n"
            f"  Receipt Total:    ${receipt_total:,.2f}\n"
            f"  Status: {match_status}\n"
            f"  Difference: ${calculated_total - receipt_total:,.2f}")

# Check each order against its receipt
print(check_order_total(df, 2742071, 152811.89))
print(check_order_total(df, 2173913, 162388.71))
print(check_order_total(df, 6128929, 923441.25))

# If there are mismatches, let's see the number of line items and sum of components
for order_id, receipt_total in [(2742071, 152811.89), (2173913, 162388.71), (6128929, 923441.25)]:
    order_lines = df[df['order_id'] == order_id]
    if not order_lines.empty and abs(order_lines['total_price'].sum() - receipt_total) >= 0.01:
        print(f"\nDetailed breakdown for Order ID {order_id}:")
        print(f"  Number of line items: {len(order_lines)}")
        print(f"  Sum of subtotals: ${order_lines['subtotal'].sum():,.2f}")
        print(f"  Sum of shipping: ${order_lines['shipping_price'].sum():,.2f}")
        print(f"  Sum of taxes: ${(order_lines['subtotal'] * 0.0925).sum():,.2f}")


Order ID 2742071:
  Calculated Total: $152,811.90
  Receipt Total:    $152,811.89
  Status: ✓ MATCH
  Difference: $0.01
Order ID 2173913:
  Calculated Total: $162,388.72
  Receipt Total:    $162,388.71
  Status: ✓ MATCH
  Difference: $0.01
Order ID 6128929:
  Calculated Total: $923,441.24
  Receipt Total:    $923,441.25
  Status: ✓ MATCH
  Difference: $-0.01


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [136]:
# How much did each of the top 5 clients by quantity spend? Check your work from Part 1 for client ids.

# Filter DataFrame by top 5 client IDs
top_clients_df = df[df['client_id'].isin(top_5_client_ids)]

# Get unique client information
client_info = top_clients_df.drop_duplicates(subset=['client_id'])[['client_id', 'first', 'last']]

# Create function to calculate total spend for each Client ID
def calculate_total_spend(client_id):
    """
    Calculate total spend for a specific client
    """
    client_data = top_clients_df[top_clients_df['client_id'] == client_id]
    total_spend = 0
    for _, row in client_data.iterrows():
        total_spend += row['qty'] * row['unit_price']
    return total_spend

# Calculate total spend for each client
total_spend = []
for client_id in client_info['client_id']:
    spend = calculate_total_spend(client_id)
    total_spend.append({'client_id': client_id, 'Total Spend': spend})

# Convert to DataFrame
total_spend_df = pd.DataFrame(total_spend)

# Merge client information with total spend
client_total_spend = pd.merge(client_info, total_spend_df, on='client_id')

# Sort by total spend in descending order
client_total_spend_sorted = client_total_spend.sort_values('Total Spend', ascending=False)

# Print the results
print("Top 5 Clients Total Spend:")
print(client_total_spend_sorted)

Top 5 Clients Total Spend:
   client_id      first     last  Total Spend
3      24741     Kendra  Garrett  70176885.25
4      38378  Alexandra    Young   8384321.15
2      66037     Angela  Everett   7995708.38
1      46820      Bryan    Myers   7317356.03
0      33615    Jessica    Reyes   5839032.11


In [137]:
#Create a summary DataFrame showing the totals for the top 5 clients with the following information:
# total units purchased, total shipping price, total revenue, and total profit.

# Filter DataFrame by top 5 clients
top_clients_df = df[df['client_id'].isin(top_5_clients)]

# Create function to calculate client summary
def calculate_client_summary(client_id):
    """
    Calculate summary metrics for a specific client
    """
    client_data = top_clients_df[top_clients_df['client_id'] == client_id]
    
    # Calculate metrics
    total_units = client_data['qty'].sum()
    total_shipping = client_data['shipping_price'].sum()
    total_revenue = client_data['total_price'].sum()
    total_profit = client_data['profit'].sum()
    
    first_name = client_data['first'].iloc[0]
    last_name = client_data['last'].iloc[0]
    
    return {
        'Client ID': client_id,
        'First Name': first_name,
        'Last Name': last_name,
        'Total Units Purchased': total_units,
        'Total Shipping Price': total_shipping,
        'Total Revenue': total_revenue,
        'Total Profit': total_profit
    }

# Calculate summary for each top client
client_summaries = []
for client_id in top_5_clients:
    client_summaries.append(calculate_client_summary(client_id))

# Create DataFrame from summaries
top_clients_summary = pd.DataFrame(client_summaries)

# Sort by Total Profit in descending order
top_clients_summary_sorted = top_clients_summary.sort_values('Total Profit', ascending=False)

# Print the results
print("Top 5 Clients Summary:")
print(top_clients_summary_sorted)

    

Top 5 Clients Summary:
   Client ID First Name Last Name  Total Units Purchased  \
4      24741     Kendra   Garrett                 239862   
3      38378  Alexandra     Young                  73667   
1      66037     Angela   Everett                  43018   
2      46820      Bryan     Myers                  75768   
0      33615    Jessica     Reyes                  64313   

   Total Shipping Price  Total Revenue  Total Profit  
4            5126448.37   8.226889e+07  3.657999e+07  
3            3429455.40   1.290655e+07  3.271830e+06  
1            1395151.85   1.025951e+07  3.255032e+06  
2            1601448.84   9.743794e+06  2.736603e+06  
0            1828984.89   8.377309e+06  2.201995e+06  


In [138]:
# Format the data and rename the columns to names suitable for presentation.

# Define the money columns
money_columns = ['Total Shipping Price', 'Total Revenue', 'Total Profit']

# Define a function that converts a dollar amount to millions
def currency_format_millions(amount):
    """
    Convert dollar amount to millions with two decimal places
    """
    return round(amount / 1_000_000, 2)

# Create a copy of the sorted summary to avoid modifying the original
top_clients_summary_formatted = top_clients_summary_sorted.copy()

# Apply the currency_format_millions function to only the money columns
for col in money_columns:
    top_clients_summary_formatted[col] = top_clients_summary_formatted[col].apply(currency_format_millions)

# Rename the columns to reflect the change in the money format
column_rename_map = {
    'Client ID': 'Client ID',
    'First Name': 'First Name',
    'Last Name': 'Last Name',
    'Total Units Purchased': 'Total Units',
    'Total Shipping Price': 'Shipping (Millions $)',
    'Total Revenue': 'Revenue (Millions $)',
    'Total Profit': 'Profit (Millions $)'
}

top_clients_summary_formatted = top_clients_summary_formatted.rename(columns=column_rename_map)

# Print the results
print("Top 5 Clients Summary (Formatted):")
print(top_clients_summary_formatted)



Top 5 Clients Summary (Formatted):
   Client ID First Name Last Name  Total Units  Shipping (Millions $)  \
4      24741     Kendra   Garrett       239862                   5.13   
3      38378  Alexandra     Young        73667                   3.43   
1      66037     Angela   Everett        43018                   1.40   
2      46820      Bryan     Myers        75768                   1.60   
0      33615    Jessica     Reyes        64313                   1.83   

   Revenue (Millions $)  Profit (Millions $)  
4                 82.27                36.58  
3                 12.91                 3.27  
1                 10.26                 3.26  
2                  9.74                 2.74  
0                  8.38                 2.20  


In [139]:
# Sort the updated data by "Total Profit (millions)" form highest to lowest and assign the sort to a new DatFrame.

top_clients_final = top_clients_summary_formatted.sort_values('Profit (Millions $)', ascending=False)

# Print the results
print("Top 5 Clients Summary (Final Sorted):")
print(top_clients_final)

Top 5 Clients Summary (Final Sorted):
   Client ID First Name Last Name  Total Units  Shipping (Millions $)  \
4      24741     Kendra   Garrett       239862                   5.13   
3      38378  Alexandra     Young        73667                   3.43   
1      66037     Angela   Everett        43018                   1.40   
2      46820      Bryan     Myers        75768                   1.60   
0      33615    Jessica     Reyes        64313                   1.83   

   Revenue (Millions $)  Profit (Millions $)  
4                 82.27                36.58  
3                 12.91                 3.27  
1                 10.26                 3.26  
2                  9.74                 2.74  
0                  8.38                 2.20  
